In [2]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [11]:
# get data
topic = "roche"

pipeline_executor = PipelineExecutor()
roche = pipeline_executor.execute(query=topic, max_articles=None, overwrite=True)

# safe and load quantum_df using parquet
roche.to_parquet('data/clean/roche.gzip', compression='gzip')
roche = pd.read_parquet('data/clean/roche.gzip')

Getting news article info: 100%|██████████| 588/588 [07:01<00:00,  1.39it/s]


In [27]:
roche = pd.read_parquet('data/clean/roche.gzip')
roche = roche.sample(n=20)
importlib.reload(ww)

# Paragraphs - Word - silhouette
wizard = ww.WordWizard(df=roche, interest = 'paragraph')
wizard.create_word_embeddings(wizard.interest)
wizard.cluster_embeddings(wizard.interest)
wizard.entitiy_recognition(wizard.interest)
wizard.summarize_medoids(wizard.interest)
wizard.find_sentiment(wizard.interest)
wizard.topic_modelling(wizard.interest)

# Paragraphs - Sentence - silhouette
wizard2 = ww.WordWizard(df=roche, interest = 'paragraph')
wizard2.create_sentence_embeddings(wizard2.interest)
wizard2.cluster_embeddings(wizard2.interest)
wizard2.entitiy_recognition(wizard2.interest)
wizard2.summarize_medoids(wizard2.interest)
wizard2.find_sentiment(wizard2.interest)
wizard2.topic_modelling(wizard2.interest)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting organizations for column paragraph: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]
Creating summaries for medoids of column paragraph: 100%|███████

KeyError: 'article'

In [29]:
# Article - Word - silhouette
wizard3 = ww.WordWizard(df=roche, interest = 'body')
wizard3.create_word_embeddings(wizard3.interest)
wizard3.cluster_embeddings(wizard3.interest)
wizard3.entitiy_recognition(wizard3.interest)
wizard3.summarize_medoids(wizard3.interest)
wizard3.find_sentiment(wizard3.interest)
wizard3.topic_modelling(wizard3.interest)

# Article - Sentence - silhouette
wizard4 = ww.WordWizard(df=roche, interest = 'body')
wizard4.create_sentence_embeddings(wizard4.interest)
wizard4.cluster_embeddings(wizard4.interest)
wizard4.entitiy_recognition(wizard4.interest)
wizard4.summarize_medoids(wizard4.interest)
wizard4.find_sentiment(wizard4.interest)
wizard4.topic_modelling(wizard4.interest)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Calculating sentiment for column body: 100%|██████████| 18/18 [01:30<00:00,  5.03s/it]


{0: [('sister', 0.0012362062942943084),
  ('baseline', 0.0012362062942943084),
  ('candace', 0.0012362062942943084),
  ('forms', 0.0012362062942943084),
  ('brain', 0.0012362062942943084),
  ('minutes', 0.0012362062942943084),
  ('toiletries', 0.0012362062942943084),
  ('relapsing', 0.0012362062942943084),
  ('did', 0.0012362062942943084),
  ('restrictions', 0.0012362062942943084),
  ('airports', 0.0012362062942943084),
  ('believe', 0.0012362062942943084),
  ('beacon', 0.0012362062942943084),
  ('mcmichael', 0.0012362062942943084),
  ('ongoing', 0.0012362062942943084),
  ('tolerance', 0.0012362062942943084),
  ('compared', 0.0012325493141377448),
  ('wear', 0.0012325493141377448),
  ('program', 0.0012325493141377448),
  ('carry', 0.0012325493141377448)],
 1: [('france', 0.0011880020236991842),
  ('mea', 0.0011880020236991842),
  ('scannell', 0.0011880020236991842),
  ('dynamics', 0.0011880020236991842),
  ('job', 0.0011880020236991842),
  ('drugsrevenue', 0.0011880020236991842),
  ('m

In [25]:
# silhouette score ain't working with too few samples I think
from sklearn.cluster import KMeans
df = wizard.df
from sklearn.metrics import silhouette_score
column = 'paragraph'
k=2
embed_column = column + wizard.EMB_SUFFIX
kk = KMeans(n_clusters=k, n_init='auto').fit(df[embed_column].tolist())
labels = kk.labels_
silhouette_score(df[embed_column].tolist(), labels, metric='euclidean')

0.18544431

In [16]:
# check the number of entities in each cluster
wizard.df['paragraph_sentence_embeddings_clusters'].value_counts()

1    8
0    3
4    2
2    2
7    1
5    1
6    1
3    1
8    1
Name: paragraph_sentence_embeddings_clusters, dtype: int64

In [17]:
# check the number of medoids per cluster
wizard.df[wizard.df['paragraph_sentence_embeddings_clusters_medoids'] == True][['paragraph_sentence_embeddings', 'paragraph_sentence_embeddings_clusters']]

,paragraph_sentence_embeddings,paragraph_sentence_embeddings_clusters
0,"[0.037035826593637466, -0.008866664953529835, ...",0
1,"[-0.029360659420490265, -0.06645254045724869, ...",4
2,"[-0.054638300091028214, -0.03934699669480324, ...",7
4,"[0.021085450425744057, -0.07643812894821167, -...",1
5,"[-0.07643575221300125, -0.07916200160980225, -...",5
6,"[0.047656454145908356, -0.03696030005812645, -...",1
8,"[0.0005669331876561046, -0.07934823632240295, ...",6
9,"[0.059580232948064804, -0.022906964644789696, ...",0
11,"[-0.05660158023238182, 0.0067533645778894424, ...",3
12,"[0.02275889180600643, -0.04268726333975792, -0...",4
